# Earth Temperature Integrator (Interactive)

In [4]:
#Import libraries
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
import matplotlib.animation as animation
import random
from matplotlib.patches import Rectangle

In [5]:
#Set up integrator

#constants
C = 2.08e8
alpha = float(input("Enter the starting albedo (between 0 and 1.0): ")) #Best = 0.32
epsilon = float(input("Enter the starting emissivity (between 0 and 1.0): ")) #Best = 0.78
sigma = 5.67e-8
S = int(input("Enter the starting solar flux: ")) #Best = 1350

#Initialize
steps = 450
T_now = (np.zeros(steps))
H_now = (np.zeros(steps) +1)
T_next = np.zeros(steps) 
H_next = np.zeros(steps)


dt = 1e6 #seconds

#Differential equation to integrate
#dH = ((1 - alpha)*S)/4 - epsilon*sigma*(T**4)

for i in range(0, steps - 1):
    H_next[i] = H_now[i] + (((1 - alpha)*S)/4 - sigma*(T_now[i]**4) + epsilon*sigma*(242.4**4)) * dt
    
    H_now[i+1] = H_next[i]
    
    T_next[i] = H_now[i] / C                       
    
    T_now[i+1] = T_next[i]
                            

Enter the starting albedo (between 0 and 1.0):  0.1
Enter the starting emissivity (between 0 and 1.0):  0.9
Enter the starting solar flux:  900


In [6]:
#Set up animation

step = []
temp = []

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
plt.title('Earth Temperature Evolution')
plt.subplots_adjust(left=0.25, bottom=0.25)

ax1.set_ylim(0,400)
ax1.set_xlim(0,500)
ax1.set_xlabel('Step')
ax1.set_ylabel('Temperature (K)')
ax1.add_patch((Rectangle((0,280), 500, 20, color = 'green', alpha = 0.2)))

def animate(i):
    step.append(i)
    temp.append(T_now[i])
    ax1.plot(step, temp, color = 'red')


ani = animation.FuncAnimation(fig, animate, interval = 40)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …